<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/EqBalancing_Xpath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from lxml import html
import re
import pandas as pd
import numpy as np

In [2]:
content=requests.get("https://en.wikipedia.org/wiki/Chemical_element").content
tree = html.fromstring(content)

In [3]:
symbol_names=tree.xpath(r'//table//td//a/span[@title]/@title')
mws = tree.xpath(r'//table//td//a//span[@data-sort-value]/../text()')

In [4]:
mws=[float(mw) for mw in mws]

In [5]:
sns=[symbol_name.replace(' ','').split(',') for symbol_name in symbol_names]
symbol, name=[list(i) for i in zip(*sns)]

In [6]:
rxn = 'aKNO3 + 5C12H22O11 -> bN2 + cCO2 + dH2O + eK2CO3'

In [7]:

left,right = rxn.replace(' ','').split('->')

atom_pat=re.compile(r'([A-Z][a-z]*)([0-9]*)')
stoic_pat = re.compile(r'^([a-z]+?|[0-9]+?)(?:[A-Z][a-z]?[0-9]*)')



def elementalize(side,df,sign=-1):
    molecules = side.split('+')
    for molecule in molecules:
        stoic = stoic_pat.findall(molecule)
        name=molecule[len(stoic):]
        if len(stoic)==0:
            n=sign
        elif stoic[0].isnumeric():
            n=int(stoic[0])*sign
        else:
            n=stoic[0]
        df.loc['stoic',name]=n
        atoms = atom_pat.findall(molecule)
        for atom, num in atoms:
            val= 1 if num=='' else int(num)
            if atom in df.index:
                if pd.isna(df.loc[atom,name]):
                    df.loc[atom,name]=val
                else:
                    df.loc[atom,name]+=val
            else:
                df.loc[atom,name]=val
    df.fillna(0,inplace=True)
    return df

def atomize(molecule):
    atoms={}
    print(molecule)

    return atoms

df=pd.DataFrame()
df= elementalize(left,df,sign=1)
df= elementalize(right,df,sign=-1)
fixed_col = pd.to_numeric(df.loc['stoic'],errors='coerce').notna()
var_col=pd.to_numeric(df.loc['stoic'],errors='coerce').isna()
fixed_sum=(df.loc[df.index[1:],fixed_col]*pd.to_numeric(df.loc['stoic',fixed_col])).sum(axis=1)
np.linalg.solve(df.loc[df.index[1:],var_col].to_numpy().astype(np.float64),fixed_sum.to_numpy())

array([-48.,  24.,  36.,  55.,  24.])